In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.lines import Line2D
from pathlib import Path
import sys
sys.path.append('dream_hpn_code_modules')
from dream_data_loader import DreamDataLoader
import pickle
# Create directories
Path('results').mkdir(exist_ok=True)
Path('figures').mkdir(exist_ok=True)

In [2]:
"""
Display results tables and generate network visualizations
Uses ORIGINAL visualization code, just saves 4 separate files
"""



print("="*80)
print("Generating tables and visualizations from cached results")
print("="*80)

# Create directories
results_dir = Path('results')
results_dir.mkdir(exist_ok=True)
figures_dir = Path('figures')
figures_dir.mkdir(exist_ok=True)

# Load cached results
protein_errors = pd.read_csv('dream_hpn_cache/protein_errors.csv')

print(f"\nLoaded {len(protein_errors)} measurements")
print(f"  Per formulation: {len(protein_errors)//3}")
print(f"  Unique test cases: {protein_errors['test_case_id'].nunique()}")

# ============================================================================
# TABLE 1: Overall Log₂ Errors
# ============================================================================
print("\n" + "="*80)
print("TABLE 1: Log₂ Errors (mean ± SE)")
print("="*80)

table1_data = []
for form in ['MASS_ACTION', 'MM', 'HILL']:
    form_df = protein_errors[protein_errors['formulation'] == form.lower()]
    n = len(form_df)
    
    b_mean = form_df['baseline_log2'].mean()
    b_se = form_df['baseline_log2'].std() / np.sqrt(n)
    o_mean = form_df['ode_log2'].mean()
    o_se = form_df['ode_log2'].std() / np.sqrt(n)
    s_mean = form_df['scm_log2'].mean()
    s_se = form_df['scm_log2'].std() / np.sqrt(n)
    
    table1_data.append({
        'Formulation': form,
        'Baseline_Mean': b_mean,
        'Baseline_SE': b_se,
        'ODE_Mean': o_mean,
        'ODE_SE': o_se,
        'SCM_Mean': s_mean,
        'SCM_SE': s_se
    })

table1 = pd.DataFrame(table1_data)
print(f"\n{table1.to_string(index=False)}")
table1.to_csv(results_dir / 'log2_errors.csv', index=False)
print(f"\n✓ Saved: {results_dir / 'log2_errors.csv'}")

# ============================================================================
# TABLE 2: Overall Fold Errors
# ============================================================================
print("\n" + "="*80)
print("TABLE 2: Fold Errors (mean ± SE)")
print("="*80)

table2_data = []
for form in ['MASS_ACTION', 'MM', 'HILL']:
    form_df = protein_errors[protein_errors['formulation'] == form.lower()]
    n = len(form_df)
    
    b_mean = form_df['baseline_fold'].mean()
    b_se = form_df['baseline_fold'].std() / np.sqrt(n)
    o_mean = form_df['ode_fold'].mean()
    o_se = form_df['ode_fold'].std() / np.sqrt(n)
    s_mean = form_df['scm_fold'].mean()
    s_se = form_df['scm_fold'].std() / np.sqrt(n)
    
    table2_data.append({
        'Formulation': form,
        'Baseline_Mean': b_mean,
        'Baseline_SE': b_se,
        'ODE_Mean': o_mean,
        'ODE_SE': o_se,
        'SCM_Mean': s_mean,
        'SCM_SE': s_se
    })

table2 = pd.DataFrame(table2_data)
print(f"\n{table2.to_string(index=False)}")
table2.to_csv(results_dir / 'fold_errors.csv', index=False)
print(f"\n✓ Saved: {results_dir / 'fold_errors.csv'}")

Generating tables and visualizations from cached results

Loaded 357 measurements
  Per formulation: 119
  Unique test cases: 17

TABLE 1: Log₂ Errors (mean ± SE)

Formulation  Baseline_Mean  Baseline_SE  ODE_Mean   ODE_SE  SCM_Mean   SCM_SE
MASS_ACTION       0.525296     0.085151  0.718587 0.081378  0.410674 0.061650
         MM       0.525296     0.085151  0.616872 0.056034  0.390171 0.054822
       HILL       0.525296     0.085151  0.702390 0.061296  0.383325 0.053340

✓ Saved: results/log2_errors.csv

TABLE 2: Fold Errors (mean ± SE)

Formulation  Baseline_Mean  Baseline_SE  ODE_Mean   ODE_SE  SCM_Mean   SCM_SE
MASS_ACTION       1.955517     0.202253  2.240240 0.263049  1.559402 0.120458
         MM       1.955517     0.202253  1.736021 0.113723  1.481356 0.097979
       HILL       1.955517     0.202253  1.880604 0.130949  1.463716 0.093442

✓ Saved: results/fold_errors.csv


In [9]:
# Draw# ============================================================================
# VISUALIZATIONS - ENHANCED VERSION
# ============================================================================
print("\n" + "="*80)
print("Generating network visualizations")
print("="*80)

# Load data
loader = DreamDataLoader('dream_hpn_data', aggregate_sites=True)
_, _, proteins = loader.load_data()
gene_pkn = loader.build_literature_pkn()
graph, _ = loader.map_to_protein_network(gene_pkn, proteins)

# Load pCOA structures
with open('dream_hpn_cache/pcoa_structures_aggTrue.pkl', 'rb') as f:
    pcoa_structures = pickle.load(f)

# =============================================================================
# HELPER FUNCTIONS (from original code)
# =============================================================================

COLORS = {
    'node_blue': '#6BAED6',
    'edge_black': '#252525',
    'edge_cycle': '#E69F00',
}

def find_cycle_edges(G):
    cycle_edges = set()
    try:
        cycles = list(nx.simple_cycles(G))
        for cycle in cycles:
            for i in range(len(cycle)):
                u = cycle[i]
                v = cycle[(i + 1) % len(cycle)]
                cycle_edges.add((u, v))
    except:
        pass
    return cycle_edges

# =============================================================================
# PKN BIOLOGICAL - ENHANCED
# =============================================================================

fig, ax = plt.subplots(figsize=(12, 10))

activation_edges = [
    ('EGFR', 'MEK'),
    ('MEK', 'MAPK'),
    ('EGFR', 'PDK1'),
    ('PDK1', 'AKT'),
    ('AKT', 'mTOR'),
    ('mTOR', 'EGFR'),
    ('MAPK', 'EGFR'),
    ('mTOR', 'PDK1'),
]

inhibition_edges = [
    ('AKT', 'GSK3'),
]

G_pkn = nx.DiGraph()
G_pkn.add_edges_from(activation_edges + inhibition_edges)

cycle_edges = find_cycle_edges(G_pkn)

cycle_activation = [e for e in activation_edges if e in cycle_edges]
regular_activation = [e for e in activation_edges if e not in cycle_edges]
cycle_inhibition = [e for e in inhibition_edges if e in cycle_edges]
regular_inhibition = [e for e in inhibition_edges if e not in cycle_edges]

pos_pkn = nx.kamada_kawai_layout(G_pkn)

# Enhanced nodes - larger, thicker borders
nx.draw_networkx_nodes(G_pkn, pos_pkn, 
                       node_color=COLORS['node_blue'], 
                       node_size=9000,
                       edgecolors=COLORS['edge_black'], 
                       linewidths=7, 
                       ax=ax)

# Enhanced edges - thicker, larger arrows
if regular_activation:
    nx.draw_networkx_edges(G_pkn, pos_pkn, edgelist=regular_activation,
                           edge_color=COLORS['edge_black'], 
                           arrows=True, arrowsize=60, arrowstyle='-|>',
                           width=7.0, style='solid',
                           connectionstyle='arc3,rad=0.15',
                           min_source_margin=60, min_target_margin=60, ax=ax)

if regular_inhibition:
    # Draw inhibition edges with dashed line for better arrowhead rendering
    nx.draw_networkx_edges(G_pkn, pos_pkn, edgelist=regular_inhibition,
                           edge_color=COLORS['edge_black'], 
                           arrows=True, arrowsize=60, arrowstyle='-|>',
                           width=7.0, style='dashed',
                           connectionstyle='arc3,rad=0.15',
                           min_source_margin=60, min_target_margin=60, ax=ax)

if cycle_activation:
    nx.draw_networkx_edges(G_pkn, pos_pkn, edgelist=cycle_activation,
                           edge_color=COLORS['edge_cycle'], 
                           arrows=True, arrowsize=60, arrowstyle='-|>',
                           width=8.0, style='solid',
                           connectionstyle='arc3,rad=0.2',
                           min_source_margin=60, min_target_margin=60, ax=ax)

if cycle_inhibition:
    # Draw cycle inhibition edges with dashed line
    nx.draw_networkx_edges(G_pkn, pos_pkn, edgelist=cycle_inhibition,
                           edge_color=COLORS['edge_cycle'], 
                           arrows=True, arrowsize=60, arrowstyle='-|>',
                           width=8.0, style='dashed',
                           connectionstyle='arc3,rad=0.2',
                           min_source_margin=60, min_target_margin=60, ax=ax)

# Enhanced labels - larger font
nx.draw_networkx_labels(G_pkn, pos_pkn, font_size=26, font_weight='bold', ax=ax)

# Enhanced legend - larger with longer lines to show patterns clearly
legend_elements = [
    Line2D([0], [0], color=COLORS['edge_black'], linewidth=6, 
           linestyle='solid', marker='>', markersize=14, label='Activation'),
    Line2D([0], [0], color=COLORS['edge_black'], linewidth=6,
           linestyle='dashed', marker='>', markersize=14, label='Inhibition'),
    Line2D([0], [0], color=COLORS['edge_cycle'], linewidth=6,
           linestyle='solid', marker='>', markersize=14, label='Feedback cycle'),
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=18, framealpha=0.95,
         borderpad=0.9, labelspacing=0.7, handlelength=4.5)

ax.margins(0.08)
ax.axis('off')

plt.tight_layout(pad=0.1)
plt.savefig(figures_dir / 'pkn.png', dpi=300, bbox_inches='tight', 
           facecolor='white', pad_inches=0.15)
print("✓ Saved: figures/pkn.png")
plt.close()

# =============================================================================
# PCOA DAGS (3 separate files) - ENHANCED
# =============================================================================

for form_name in ['mass_action', 'mm', 'hill']:
    structure, _ = pcoa_structures[form_name]
    
    # Filter out I_MEK from structure
    structure_filtered = {}
    for var, parents in structure.items():
        structure_filtered[var] = [p for p in parents if p != 'I_MEK']
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    pos_pcoa = {
        'EGFR': (0, 3),
        'AKT': (2, 3),
        'GSK3': (4, 3),
        'MEK': (0, 1),
        'PDK1': (2, 1),
        'MAPK': (4, 1),
        'mTOR': (6, 2),
        'I_AKT': (1, -0.5),
    }
    
    # Build edges from filtered structure
    protein_edges = []
    inhibition_edges = []
    
    for child, parents in structure_filtered.items():
        for parent in parents:
            if parent == 'I_AKT':
                inhibition_edges.append((parent, child))
            elif parent in proteins:
                protein_edges.append((parent, child))
    
    G_pcoa = nx.DiGraph()
    G_pcoa.add_edges_from(protein_edges)
    G_pcoa.add_edges_from(inhibition_edges)
    
    # Find cycles (protein edges only)
    protein_only_G = nx.DiGraph()
    protein_only_G.add_edges_from(protein_edges)
    cycle_edges_pcoa = find_cycle_edges(protein_only_G)
    regular_edges_pcoa = [e for e in protein_edges if e not in cycle_edges_pcoa]
    
    # Enhanced protein nodes - larger with thicker borders
    protein_nodes = [p for p in proteins if p in G_pcoa]
    nx.draw_networkx_nodes(G_pcoa, pos_pcoa, nodelist=protein_nodes,
                           node_color='lightblue', node_shape='o',
                           node_size=12000, ax=ax, edgecolors='black', linewidths=8)
    
    # Enhanced inhibition nodes - larger with thicker borders
    inhibition_nodes = [n for n in G_pcoa.nodes() if n == 'I_AKT']
    if inhibition_nodes:
        nx.draw_networkx_nodes(G_pcoa, pos_pcoa, nodelist=inhibition_nodes,
                               node_color='#FFE44D', node_shape='s',
                               node_size=16000, ax=ax, edgecolors='black', linewidths=8)
    
    # Enhanced labels - larger text
    nx.draw_networkx_labels(G_pcoa, pos_pcoa, font_size=28, font_weight='bold', ax=ax)
    
    # Enhanced edges - thicker, larger arrows
    # Separate PDK1 → mTOR to give it negative curvature
    pdk_to_mtor = [(u, v) for u, v in regular_edges_pcoa if u == 'PDK1' and v == 'mTOR']
    other_regular = [(u, v) for u, v in regular_edges_pcoa if not (u == 'PDK1' and v == 'mTOR')]
    
    if other_regular:
        nx.draw_networkx_edges(G_pcoa, pos_pcoa, edgelist=other_regular,
                              edge_color='black', arrows=True, arrowsize=70,
                              arrowstyle='-|>', width=8.0, ax=ax,
                              connectionstyle='arc3,rad=0.2',
                              min_source_margin=70, min_target_margin=70)
    
    if pdk_to_mtor:
        nx.draw_networkx_edges(G_pcoa, pos_pcoa, edgelist=pdk_to_mtor,
                              edge_color='black', arrows=True, arrowsize=70,
                              arrowstyle='-|>', width=8.0, ax=ax,
                              connectionstyle='arc3,rad=-0.2',
                              min_source_margin=70, min_target_margin=70)
    
    # Enhanced cycle edges - thicker
    if cycle_edges_pcoa:
        nx.draw_networkx_edges(G_pcoa, pos_pcoa, edgelist=list(cycle_edges_pcoa),
                              edge_color='red', arrows=True, arrowsize=70,
                              arrowstyle='-|>', width=9.0, ax=ax,
                              connectionstyle='arc3,rad=0.3',
                              min_source_margin=70, min_target_margin=70)
    
    # Enhanced inhibition edges - thicker
    if inhibition_edges:
        # Separate I_AKT → mTOR and I_AKT → GSK3 for special handling
        i_akt_to_mtor = [(u, v) for u, v in inhibition_edges if v == 'mTOR']
        i_akt_to_gsk3 = [(u, v) for u, v in inhibition_edges if v == 'GSK3']
        other_inhib = [(u, v) for u, v in inhibition_edges if v not in ['mTOR', 'GSK3']]
        
        # Draw I_AKT → mTOR with strong negative curve to avoid MAPK
        if i_akt_to_mtor:
            nx.draw_networkx_edges(G_pcoa, pos_pcoa, edgelist=i_akt_to_mtor,
                                  edge_color='black', arrows=True, arrowsize=70,
                                  arrowstyle='-|>', width=8.0, ax=ax,
                                  connectionstyle='arc3,rad=0.4',
                                  min_source_margin=70, min_target_margin=70)
        
        # Draw I_AKT → GSK3 with more curvature
        if i_akt_to_gsk3:
            nx.draw_networkx_edges(G_pcoa, pos_pcoa, edgelist=i_akt_to_gsk3,
                                  edge_color='black', arrows=True, arrowsize=70,
                                  arrowstyle='-|>', width=8.0, ax=ax,
                                  connectionstyle='arc3,rad=0.35',
                                  min_source_margin=70, min_target_margin=70)
        
        # Draw other inhibition edges normally
        if other_inhib:
            nx.draw_networkx_edges(G_pcoa, pos_pcoa, edgelist=other_inhib,
                                  edge_color='black', arrows=True, arrowsize=70,
                                  arrowstyle='-|>', width=8.0, ax=ax,
                                  min_source_margin=70, min_target_margin=70)
    
    ax.axis('off')
    ax.set_xlim(-0.8, 7.0)  # More space on left to prevent cutoff
    ax.set_ylim(-1.0, 3.5)  # Tighter margins
    
    plt.tight_layout(pad=0.05)
    save_path = figures_dir / f'pcoa_{form_name}.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    print(f"✓ Saved: {save_path}")
    plt.close()

print("\n" + "="*80)
print("✓ Complete!")
print("="*80)
print("\nGenerated files:")
print("  results/log2_errors.csv")
print("  results/fold_errors.csv")
print("  figures/pkn.png")
print("  figures/pcoa_mass_action.png")
print("  figures/pcoa_mm.png")
print("  figures/pcoa_hill.png")


Generating network visualizations
✓ Saved: figures/pkn.png
✓ Saved: figures/pcoa_mass_action.png
✓ Saved: figures/pcoa_mm.png
✓ Saved: figures/pcoa_hill.png

✓ Complete!

Generated files:
  results/log2_errors.csv
  results/fold_errors.csv
  figures/pkn.png
  figures/pcoa_mass_action.png
  figures/pcoa_mm.png
  figures/pcoa_hill.png
